In [2]:
import os
from keras.models import Model
from keras.optimizers import Adam
from keras.applications.vgg16 import VGG16, preprocess_input

from tensorflow.keras.preprocessing.image import ImageDataGenerator

from keras.layers import Dense,Dropout,Flatten
from keras.callbacks import ModelCheckpoint,EarlyStopping
from pathlib import Path
import numpy as np

In [3]:
df_dir=r'C:/Users/Mohammed Arif/Downloads/cats_dogs'
df_dir

'C:/Users/Mohammed Arif/Downloads/cats_dogs'

In [30]:
img_width,img_height=224,224
batch_size=5
epochs=10

In [32]:
datagen=ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

In [34]:
train_generator=datagen.flow_from_directory(
    df_dir,
    target_size=(img_width,img_height),
    batch_size=batch_size,
    class_mode='binary',
    subset='training')

Found 92 images belonging to 2 classes.


In [35]:
images,labels=next(train_generator)
print(images.shape)

(5, 224, 224, 3)


In [37]:
validation_generator=datagen.flow_from_directory(
    df_dir,
    target_size=(img_width,img_height),
    batch_size=batch_size,
    class_mode='binary',
    subset='validation'
)

Found 22 images belonging to 2 classes.


In [38]:
vgg=VGG16(input_shape=(224,224,3),include_top=False,weights='imagenet')

In [40]:
for layer in vgg.layers:
    layer.trainable=False

In [41]:
from glob import glob
folders=glob(r'C:/Users/Mohammed Arif/Downloads/cats_dogs')

In [42]:
x=Flatten()(vgg.output)
x=Dense(1000,activation='relu')(x)
prediction=Dense(len(folders),activation='softmax')(x)

#create a model object
model=Model(inputs=vgg.input,outputs=prediction)
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1000)           │    25,089,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │         1,001 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 39,804,689 (151.84 MB)

 Trainable params: 25,090,001 (95.71 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [47]:
model.compile(loss='categorical_crossentropy',
         optimizer='adam',
         metrics=['accuracy'])


In [49]:
r = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=5,
    steps_per_epoch=len(train_generator),
    validation_steps=len(validation_generator)
)


Epoch 1/5


C:\Users\Mohammed Arif\anaconda3\Lib\site-packages\keras\src\losses\losses.py:27: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return self.fn(y_true, y_pred, **self._fn_kwargs)


19/19 ━━━━━━━━━━━━━━━━━━━━ 56s 3s/step - accuracy: 0.5146 - loss: 0.0000e+00 - val_accuracy: 0.5455 - val_loss: 0.0000e+00
Epoch 2/5
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 3/5


C:\Users\Mohammed Arif\anaconda3\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


19/19 ━━━━━━━━━━━━━━━━━━━━ 72s 2s/step - accuracy: 0.5708 - loss: 0.0000e+00 - val_accuracy: 0.5455 - val_loss: 0.0000e+00
Epoch 4/5
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 5/5
19/19 ━━━━━━━━━━━━━━━━━━━━ 51s 3s/step - accuracy: 0.4811 - loss: 0.0000e+00 - val_accuracy: 0.5455 - val_loss: 0.0000e+00


In [50]:
model.compile(
    loss='categorical_crossentropy',  # Corrected the typo here
    optimizer=Adam(0.0001),
    metrics=['accuracy']
)


In [53]:
# Step 8: Evaluate the model
loss, accuracy = model.evaluate(validation_generator)
print(f'Validation Accuracy: {accuracy*100:.2f}%')


5/5 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - accuracy: 0.5013 - loss: 0.0000e+00
Validation Accuracy: 54.55%
